In [1]:
def load_csv_data(file_paths):
    data_storage = {}
    for name, file_path in file_paths.items():
        try:
            data_storage[name] = pd.read_csv(file_path, dtype=object)
            print(f"Data loaded from {file_path}")
        except FileNotFoundError:
            print(f"File {file_path} not found")
    return data_storage

def display_dataframes(data_storage):
    for name, df in data_storage.items():
        print(f"\n{name}:")
        print(df.head())

def main():
    # Define file paths
    file_paths = {
        'echonest': '../fma_metadata/echonest.csv',
        'features': '../fma_metadata/features.csv',
        'genres': '../fma_metadata/genres.csv',
        'raw_albums': '../fma_metadata/raw_albums.csv',
        'raw_artists': '../fma_metadata/raw_artists.csv',
        'raw_echonest': '../fma_metadata/raw_echonest.csv',
        'raw_genres': '../fma_metadata/raw_genres.csv',
        'raw_tracks': '../fma_metadata/raw_tracks.csv',
        'tracks': '../fma_metadata/tracks.csv'
    }

    # Load CSV files into DataFrames
    data_storage = load_csv_data(file_paths)

    # Display DataFrames
    display_dataframes(data_storage)

if __name__ == "__main__":
    main()


Read ../fma_metadata/echonest.csv
Read ../fma_metadata/features.csv
Read ../fma_metadata/genres.csv
Read ../fma_metadata/raw_albums.csv
Read ../fma_metadata/raw_artists.csv
Read ../fma_metadata/raw_echonest.csv
Read ../fma_metadata/raw_genres.csv
Read ../fma_metadata/raw_tracks.csv
Read ../fma_metadata/tracks.csv

echonest:
  Unnamed: 0        echonest      echonest.1      echonest.2  \
0        NaN  audio_features  audio_features  audio_features   
1        NaN    acousticness    danceability          energy   
2   track_id             NaN             NaN             NaN   
3          2    0.4166752327    0.6758939853    0.6344762684   
4          3    0.3744077685    0.5286430621    0.8174611317   

         echonest.3      echonest.4      echonest.5      echonest.6  \
0    audio_features  audio_features  audio_features  audio_features   
1  instrumentalness        liveness     speechiness           tempo   
2               NaN             NaN             NaN             NaN   
3    

In [5]:
import os
import random

def select_audio_files(folder_path, num_files):
    selected_files = []
    all_audio_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".mp3"):
                all_audio_files.append(os.path.join(root, file))
    if len(all_audio_files) <= num_files:
        return all_audio_files
    else:
        selected_files = random.sample(all_audio_files, num_files)
        return selected_files

def select_random_subfolders(folder_path, num_subfolders):
    return random.sample(os.listdir(folder_path), num_subfolders)

def main():
    fma_large_folder = "../fma_large1/fma_large"
    num_subfolders = 10
    num_files_per_subfolder = 100
    selected_audio_files = []

    subfolders = select_random_subfolders(fma_large_folder, num_subfolders)
    for subfolder in subfolders:
        subfolder_path = os.path.join(fma_large_folder, subfolder)
        selected_audio_files.extend(select_audio_files(subfolder_path, num_files_per_subfolder))

    # Print the number of selected audio files
    print(f"Total selected audio files: {len(selected_audio_files)}")

if __name__ == "__main__":
    main()


Total selected audio files: 1901


In [7]:
import numpy as np

def preprocess_features(all_features, fixed_length):
    all_features_fixed_length = []
    for features in all_features:
        if len(features) < fixed_length:
            # Creating a copy of the original features
            padded_features = np.array(features)
            # Appending zeros to the end to pad to fixed length
            padded_features = np.concatenate((padded_features, np.zeros(fixed_length - len(features))))
            all_features_fixed_length.append(padded_features)
        elif len(features) > fixed_length:
            # Truncating features to fixed length
            truncated_features = np.array(features[:fixed_length])
            all_features_fixed_length.append(truncated_features)
        else:
            # Keeping the features unchanged if they match the fixed length
            all_features_fixed_length.append(np.array(features))
    return np.array(all_features_fixed_length)

def main():
    # Example usage:
    # Assume all_features is a list of arrays with varying lengths
    all_features = [[1, 2, 3], [4, 5], [6, 7, 8, 9, 10]]
    # Define the desired fixed length for features
    fixed_length = 4

    all_features_np = preprocess_features(all_features, fixed_length)

    # Print the shape of the NumPy array
    print(all_features_np.shape)

if __name__ == "__main__":
    main()


(2661, 1000)


In [8]:
import os
import librosa
import numpy as np

def extract_audio_features(directory_path, duration=30):
    selected_audio_files = []

    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith(".mp3"):
                selected_audio_files.append(os.path.join(root, file))

    print(f"Total audio files selected: {len(selected_audio_files)}")

    all_features = []

    current_subfolder = None

    # Extract features from each audio file
    for audio_file in selected_audio_files:
        # Get the subfolder name
        subfolder = os.path.basename(os.path.dirname(audio_file))

        # If subfolder changes, print a message
        if current_subfolder != subfolder:
            if current_subfolder:
                print(f"|----Features extracted for {current_subfolder} folder----|")
            current_subfolder = subfolder

        features = extract_features(audio_file, duration)
        if features is not None:
            all_features.append(features)

    # Print the final message for the last subfolder
    if current_subfolder:
        print(f"|----Features extracted for {current_subfolder} folder----|")

    # Convert the list of features to a numpy array
    all_features = np.array(all_features)

    # Print the size of all_features
    print(f"Features of {len(all_features)} files have been extracted successfully.")

def extract_features(file_path, duration=30):
    try:
        # Load audio file
        y, sr = librosa.load(file_path, duration=duration)
        
        # Extract audio features
        def extract_audio_features(y, sr):
            mfcc = librosa.feature.mfcc(y=y, sr=sr)
            spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
            zero_crossing_rate = librosa.feature.zero_crossing_rate(y)
            
            # Limit MFCC features to first 100 coefficients
            mfcc_flat = mfcc.flatten()[:100]
            
            return np.concatenate((mfcc_flat, spectral_centroid.flatten(), zero_crossing_rate.flatten()))
        
        return extract_audio_features(y, sr)
    except Exception as e:
        return None

def main():
    directory_path = "../fma_large1/fma_large"
    extract_audio_features(directory_path)

if __name__ == "__main__":
    main()


Total selected audio files: 2670
|----Features extracted for fma_large1/000 folder----|
|----Features extracted for fma_large1/001 folder----|


C:\Users\bizzc\AppData\Local\Temp\ipykernel_119656\1658681598.py:8: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, duration=30)


|----Features extracted for fma_large1/002 folder----|
|----Features extracted for fma_large1/003 folder----|
|----Features extracted for fma_large1/004 folder----|
|----Features extracted for fma_large1/005 folder----|
|----Features extracted for fma_large1/006 folder----|
|----Features extracted for fma_large1/007 folder----|
|----Features extracted for fma_large1/008 folder----|
|----Features extracted for fma_large1/009 folder----|
|----Features extracted for fma_large1/010 folder----|
|----Features extracted for fma_large1/011 folder----|
|----Features extracted for fma_large1/012 folder----|
|----Features extracted for fma_large1/013 folder----|


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2661,) + inhomogeneous part.

In [10]:
# Importing required libraries
import os
from pymongo import MongoClient
import librosa
import numpy as np

def combine_and_flatten_features(mfcc, spectral_centroid, zero_crossing_rate):
    # Calculating the size of each feature array
    mfcc_length = mfcc.size
    spectral_centroid_length = spectral_centroid.size
    zero_crossing_rate_length = zero_crossing_rate.size
    
    # Finding the maximum length among the feature arrays
    max_length = max(mfcc_length, spectral_centroid_length, zero_crossing_rate_length)
    
    # Padding each feature array with zeros to match the maximum length
    padded_mfcc = np.pad(mfcc.flatten(), (0, max_length - mfcc_length))
    padded_spectral_centroid = np.pad(spectral_centroid.flatten(), (0, max_length - spectral_centroid_length))
    padded_zero_crossing_rate = np.pad(zero_crossing_rate.flatten(), (0, max_length - zero_crossing_rate_length))
    
    # Combining the padded feature arrays
    combined_features = np.hstack((padded_mfcc, padded_spectral_centroid, padded_zero_crossing_rate))
    
    return combined_features

def extract_features(file_path, duration=30):
    try:
        # Load audio file
        y, sr = librosa.load(file_path, duration=duration)
        
        # Extract features
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
        zero_crossing_rate = librosa.feature.zero_crossing_rate(y)
        
        # Combine and flatten features
        features = combine_and_flatten_features(mfcc, spectral_centroid, zero_crossing_rate)

        return features
    except Exception as e:
        return None

def main():
    # MongoDB client setup
    client = MongoClient('mongodb://localhost:27017/')
    db = client['spotify_db']  
    collection = db['audio_features']  

    # File traversal
    fma_large_folder = "../fma_large1/fma_large"
    selected_audio_files = [os.path.join(root, file) for root, _, files in os.walk(fma_large_folder) for file in files if file.endswith(".mp3")]

    print(f"Total selected audio files: {len(selected_audio_files)}")

    all_features = []
    current_subfolder = None

    for audio_file in selected_audio_files:
        subfolder = os.path.basename(os.path.dirname(audio_file))
        if current_subfolder != subfolder:
            if current_subfolder:
                print(f"|----Features extracted for fma_large1/{current_subfolder} folder----|")
            current_subfolder = subfolder

        features = extract_features(audio_file)
        if features is not None:
            # Convert features to list for MongoDB storage
            features_list = features.tolist()
            
            # Insert document into MongoDB
            collection.insert_one({'file_path': audio_file, 'features': features_list})
            all_features.append(features)

    if current_subfolder:
        print(f"|----Features extracted for fma_large1/{current_subfolder} folder----|")

    print(f"Features of {len(all_features)} files have been extracted and stored in MongoDB successfully.")

if __name__ == "__main__":
    main()


Total selected audio files: 2670
|----Features extracted for fma_large1/000 folder----|
|----Features extracted for fma_large1/001 folder----|


C:\Users\bizzc\AppData\Local\Temp\ipykernel_119656\3374685405.py:10: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, duration=30)
C:\Users\bizzc\AppData\Roaming\Python\Python311\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


|----Features extracted for fma_large1/002 folder----|
|----Features extracted for fma_large1/003 folder----|
|----Features extracted for fma_large1/004 folder----|
|----Features extracted for fma_large1/005 folder----|
|----Features extracted for fma_large1/006 folder----|
|----Features extracted for fma_large1/007 folder----|
|----Features extracted for fma_large1/008 folder----|
|----Features extracted for fma_large1/009 folder----|
|----Features extracted for fma_large1/010 folder----|
|----Features extracted for fma_large1/011 folder----|
|----Features extracted for fma_large1/012 folder----|
|----Features extracted for fma_large1/013 folder----|
Features of 2661 files have been extracted and stored in MongoDB successfully.


In [9]:
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from surprise import Dataset, Reader, SVD
import numpy as np

# MongoDB client setup
client = MongoClient('mongodb://localhost:27017/')
db = client['spotify_db']
collection = db['audio_features']

# Fetch data from MongoDB and preprocess it
cursor = collection.find({})
data = [(doc['features'][0], doc['file_path']) for doc in cursor]

# Split features and file paths
features, file_paths = zip(*data)

# Generate pseudo item IDs
item_ids = np.arange(len(features))

# Create a pandas DataFrame
df = pd.DataFrame({'item_id': item_ids, 'features': features, 'file_path': file_paths})

# Split data into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Define a Reader object specifying the rating scale
reader = Reader(rating_scale=(0, 5))

# Load data into Surprise dataset
data_surprise = Dataset.load_from_df(train_df[['item_id', 'features']], reader)

# Train the model using Singular Value Decomposition (SVD)
trainset = data_surprise.build_full_trainset()
algo = SVD()
algo.fit(trainset)

# Make predictions on the test set
testset = test_df['item_id'].tolist()
predictions = [algo.predict(uid=None, iid=item_id) for item_id in testset]

# Extract actual ratings and predicted ratings
actual_ratings = test_df['features'].apply(lambda x: x[0]).tolist()
predicted_ratings = [pred.est for pred in predictions]

# Compute RMSE (Root Mean Squared Error)
rmse = np.sqrt(mean_squared_error(actual_ratings, predicted_ratings))
print(f"Root Mean Squared Error (RMSE): {rmse}")


RMSE: 366.6184


366.6183896925667

MAE:  325.7235
MAE:  339.1736
MAE:  345.3486


c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_ranking.py:1807: FutureWarning: ndcg_score should not be used on negative y_true values. ndcg_score will raise a ValueError on negative y_true values starting from version 1.4.
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_ranking.py:1807: FutureWarning: ndcg_score should not be used on negative y_true values. ndcg_score will raise a ValueError on negative y_true val

MAE:  324.9372
MAE:  334.7091
Average MAE: 333.97841690271036
Average Precision: 0.0
Average Recall: 0.0
Average F1 Score: 0.0
Average NDCG: 1.1189861943909838


c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_ranking.py:1807: FutureWarning: ndcg_score should not be used on negative y_true values. ndcg_score will raise a ValueError on negative y_true values starting from version 1.4.
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Python311\Lib\site-packages\sklearn\metrics\_ranking.py:1807: FutureWarning: ndcg_score should not be used on negative y_true values. ndcg_score will raise a ValueError on negative y_true val

In [20]:
import os
from pymongo import MongoClient
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

def preprocess_features(df, max_features_len):
    # Pad features with zeros if necessary
  
    return features_scaled

def load_data_from_mongodb(collection):
  
def train_and_evaluate_model(train_dataset, test_dataset, max_features_len, epochs=10):
  
def main():
    # Initialize MongoDB client and collection
    client = MongoClient('mongodb://localhost:27017/')
    db = client['spotify_db']
    collection = db['audio_features']

    # Load data from MongoDB
    df, max_features_len = load_data_from_mongodb(collection)

    # Split data into training and testing sets
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

    # Preprocess training and testing data
    train_features_scaled = preprocess_features(train_df, max_features_len)
    test_features_scaled = preprocess_features(test_df, max_features_len)

    # Convert data to TensorFlow Datasets
    train_dataset = tf.data.Dataset.from_tensor_slices((train_features_scaled, train_features_scaled)).batch(64)
    test_dataset = tf.data.Dataset.from_tensor_slices((test_features_scaled, test_features_scaled)).batch(64)

    # Train and evaluate the model
    train_and_evaluate_model(train_dataset, test_dataset, max_features_len)

if __name__ == "__main__":
    main()

Epoch 1/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 92s 31ms/step - loss: 0.1364
Epoch 2/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.1343
Epoch 3/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.1343
Epoch 4/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.1343
Epoch 5/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.1343
Epoch 6/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.1343
Epoch 7/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.1343
Epoch 8/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.1343
Epoch 9/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.1343
Epoch 10/10
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.1343
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1871 
Test Loss: 0.17238038778305054
